In [1]:
import apache_beam as beam

In [2]:
def SplitRow(element):
    return element.split(',')

In [11]:
with beam.Pipeline() as p:

    input_collection = (
        p
        | "Read text from a file" >> beam.io.ReadFromText('data/dept_data.txt')
        | "Split rows" >> beam.Map(SplitRow)

    )

    accounts_count = (

        input_collection
        | "Get all accounts dept persons" >> beam.Filter(lambda record: record[3] == 'Accounts')
        | "Pair each accounts employee with 1" >> beam.Map(lambda record: ("Accounts, " + record[1],1))
        | "Group and sum Accounts" >> beam.CombinePerKey(sum)
        | "Write results for account" >> beam.io.WriteToText('data/output_branch_pipeline_accounts')
    )

    hr_count = (

        input_collection
        | "Get all HR dept persons" >> beam.Filter(lambda record: record[3] == 'HR')
        | "Pair each HR employee with 1" >> beam.Map(lambda record: ("HR, " + record[1],1))
        | "Group and sum HR" >> beam.CombinePerKey(sum)
        | "Write results for hr" >> beam.io.WriteToText('data/output_branch_pipeline_hr')
    )
    outputs = (
        (accounts_count, hr_count)
        | beam.Flatten()
        | beam.io.WriteToText('data/output_branch_pipeline_both')
    )

#### Nota:
- Si dejamos las dos outputs en cada PCollection, se generaran dos archivos separados
- Si no ponemos las dos outputs, la PColleccion output genera un merge de ambas PCollections

In [12]:
!head -n 10 data/output_branch_pipeline_accounts-00000-of-00001

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)


In [13]:
!head -n 10 data/output_branch_pipeline_hr-00000-of-00001

('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [14]:
!head -n 10 data/output_branch_pipeline_both-00000-of-00001

data/output_branch_pipeline_hr-00000-of-00001
data/output_branch_pipeline_accounts-00000-of-00001
